In [14]:
import pandas as pd 
import ast
import matplotlib.pyplot as plt

In [15]:



cust=pd.read_csv('cust.csv')
columns_to_convert = ["Quantity", "category", "products"]
for column in columns_to_convert:
    cust[column] = cust[column].apply(ast.literal_eval)
cust.head()

,Unnamed: 0,CustomerID,Quantity,NoOfTransactions,category,products,Region,SignupDate,amount,AvgAmountPerTransaction,AvgQuantityPerTransaction,FavouriteCategory,FavouriteProduct
0,0,C0001,"{2: 3, 3: 2}",5,"{'Books': 1, 'Home Decor': 1, 'Electronics': 3}","{'SoundWave Cookbook': 1, 'HomeSense Wall Art'...",South America,2022-07-10,1389,277.80,2,Electronics,SoundWave Cookbook
1,1,C0002,"{2: 3, 4: 1}",4,"{'Home Decor': 2, 'Clothing': 2}","{'BookWorld Cookware Set': 1, 'BookWorld Rug':...",Asia,2022-02-13,834,208.50,2,Home Decor,BookWorld Cookware Set
2,2,C0003,"{3: 2, 4: 2}",4,"{'Home Decor': 2, 'Clothing': 1, 'Electronics'...","{'ActiveWear Cookware Set': 1, 'ActiveWear Rug...",South America,2024-03-07,781,195.25,3,Home Decor,ActiveWear Cookware Set
3,3,C0004,"{4: 3, 3: 2, 2: 2, 1: 1}",8,"{'Books': 3, 'Home Decor': 3, 'Electronics': 2}","{'TechPro Textbook': 1, 'TechPro Rug': 1, 'Tec...",South America,2022-10-09,1920,240.00,2,Books,TechPro Textbook
4,4,C0005,"{3: 1, 2: 2}",3,"{'Home Decor': 1, 'Electronics': 2}","{'ActiveWear Cookware Set': 1, 'TechPro Smartw...",Asia,2022-08-15,873,291.00,2,Electronics,ActiveWear Cookware Set


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_similarity(cust1, cust2, cust_data):
    try:
        customer1 = cust_data[cust_data['CustomerID'] == cust1].iloc[0].to_dict()
        customer2 = cust_data[cust_data['CustomerID'] == cust2].iloc[0].to_dict()
    except IndexError:
        raise ValueError("Invalid Customer IDs provided.")
    
    #building fixed size vectors for only 4 given categories
    all_categories = list(set().union(*cust_data['category'].apply(lambda x: x.keys())))
    category1 = np.array([customer1['category'].get(cat, 0) for cat in all_categories])
    category2 = np.array([customer2['category'].get(cat, 0) for cat in all_categories])

    #cosine similarity for category score
    category_similarity = cosine_similarity([category1], [category2])[0][0]

    # # Product similarity (Jaccard similarity)
    # products1=set(customer1['products'].keys())
    # products2 =set(customer2['products'].keys())
    # product_intersection=len(products1 & products2)
    # product_union=len(products1 | products2)
    # product_similarity=product_intersection / product_union if product_union != 0 else 0
  

    #product similarity seems more subtle, so ignored

    region_similarity = 1 if customer1['Region'] == customer2['Region'] else 0

    noot_diff=abs(customer1['NoOfTransactions'] - customer2['NoOfTransactions'])
    avg_trans_diff=abs(customer1['AvgAmountPerTransaction'] - customer2['AvgAmountPerTransaction'])
    avg_qty_diff=abs(customer1['AvgQuantityPerTransaction'] - customer2['AvgQuantityPerTransaction'])

    max_noot=cust_data['NoOfTransactions'].max()
    max_avg_trans=cust_data['AvgAmountPerTransaction'].max()
    max_avg_qty=cust_data['AvgQuantityPerTransaction'].max()

    noot_similarity=1-(noot_diff / max_noot)
    avg_trans_similarity=1-(avg_trans_diff / max_avg_trans)
    avg_qty_similarity=1-(avg_qty_diff / max_avg_qty)

    weights = {
        'category': 0.3,
        # 'product': 0.1,
        'region': 0.1,
        'noot': 0.2,
        'avg_trans': 0.2,
        'avg_qty': 0.2,
    }
    overall_similarity = (
        weights['category'] * category_similarity +
        # weights['product'] * product_similarity +
        weights['region'] * region_similarity +
        weights['noot'] * noot_similarity +
        weights['avg_trans'] * avg_trans_similarity +
        weights['avg_qty'] * avg_qty_similarity
    )

    return overall_similarity


In [ ]:
similarities = {}
for i in range(1, 21):
    cust1= f"C{str(i).zfill(4)}"
    similarities[cust1] = []
    for j in range(1, 201): 
        if i!=j:
            cust2=f"C{str(j).zfill(4)}"  
            try:
                similarity_score=calculate_similarity(cust1, cust2, cust)
                similarities[cust1].append((cust2, similarity_score))
            except ValueError as e:
                print(f"Error with customer {cust2}: {e}")
                continue
    similarities[cust1].sort(key=lambda x: x[1], reverse=True)
    similarities[cust1] = similarities[cust1][:3]


Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with customer C0180: Invalid Customer IDs provided.
Error with cus

In [26]:
similarity_data = []
for cust1,matches in similarities.items():
    temp=[cust1]
    for match in matches:
        cust2, score=match
        temp.append(cust2)
        temp.append('%.3f'%score)
    similarity_data.append(temp)
_similarities = pd.DataFrame(similarity_data, columns=['CustomerID','CustomerID_1','CustomerID_1_score','CustomerID_2','CustomerID_2_score','CustomerID_3','CustomerID_3_score'])

_similarities.head()

,CustomerID,CustomerID_1,CustomerID_1_score,CustomerID_2,CustomerID_2_score,CustomerID_3,CustomerID_3_score
0,C0001,C0190,0.950,C0048,0.942,C0192,0.937
1,C0002,C0106,0.953,C0159,0.952,C0178,0.951
2,C0003,C0129,0.938,C0031,0.934,C0039,0.926
3,C0004,C0113,0.979,C0012,0.949,C0104,0.936
4,C0005,C0007,0.977,C0140,0.965,C0146,0.938


In [27]:
_similarities.to_csv('lookilike.csv')